--- Day 20: Particle Swarm ---

Suddenly, the GPU contacts you, asking for help. Someone has asked it to simulate too many particles, and it won't be able to finish them all in time to render the next frame at this rate.

It transmits to you a buffer (your puzzle input) listing each particle in order (starting with particle 0, then particle 1, particle 2, and so on). For each particle, it provides the X, Y, and Z coordinates for the particle's position (p), velocity (v), and acceleration (a), each in the format <X,Y,Z>.

Each tick, all particles are updated simultaneously. A particle's properties are updated in the following order:

```
Increase the X velocity by the X acceleration.
Increase the Y velocity by the Y acceleration.
Increase the Z velocity by the Z acceleration.
Increase the X position by the X velocity.
Increase the Y position by the Y velocity.
Increase the Z position by the Z velocity.
```

Because of seemingly tenuous rationale involving z-buffering, the GPU would like to know which particle will stay closest to position `<0,0,0>` in the long term. Measure this using the Manhattan distance, which in this situation is simply the sum of the absolute values of a particle's X, Y, and Z position.

For example, suppose you are only given two particles, both of which stay entirely on the X-axis (for simplicity). Drawing the current states of particles 0 and 1 (in that order) with an adjacent a number line and diagram of current X positions (marked in parenthesis), the following would take place:

```
p=< 3,0,0>, v=< 2,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=< 4,0,0>, v=< 0,0,0>, a=<-2,0,0>                         (0)(1)

p=< 4,0,0>, v=< 1,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=< 2,0,0>, v=<-2,0,0>, a=<-2,0,0>                      (1)   (0)

p=< 4,0,0>, v=< 0,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=<-2,0,0>, v=<-4,0,0>, a=<-2,0,0>          (1)               (0)

p=< 3,0,0>, v=<-1,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=<-8,0,0>, v=<-6,0,0>, a=<-2,0,0>                         (0)   
```

At this point, particle 1 will never be closer to <0,0,0> than particle 0, and so, in the long run, particle 0 will stay closest.

Which particle will stay closest to position <0,0,0> in the long term?

In [117]:
test_particles = [
    "p=<3,0,0>, v=<2,0,0>, a=<-1,0,0>",
    "p=<4,0,0>, v=<0,0,0>, a=<-2,0,0>"
]

In [100]:
def particle_struct(particle):
    components = particle.split(", ")
    position = [int(x) for x in components[0].split("=")[1].strip("<").strip(">").split(",")]
    velocity = [int(x) for x in components[1].split("=")[1].strip("<").strip(">").split(",")]
    accel = [int(x) for x in components[2].split("=")[1].strip("<").strip(">").split(",")]
    return {"p":position, "v":velocity, "a":accel}

In [101]:
def update_particle(particle):
    for i in range(3):
        particle['v'][i] += particle['a'][i]
    for j in range(3):
        particle['p'][j] += particle['v'][j]
    particle['d'] = sum([abs(x) for x in particle['p']])
    return particle

In [118]:
test_particles = [particle_struct(x) for x in test_particles]
test_particles

[{'a': [-1, 0, 0], 'p': [3, 0, 0], 'v': [2, 0, 0]},
 {'a': [-2, 0, 0], 'p': [4, 0, 0], 'v': [0, 0, 0]}]

In [119]:
for i in range(3):
    print("Step:", i)
    [update_particle(x) for x in test_particles]
    print(test_particles)

Step: 0
[{'p': [4, 0, 0], 'v': [1, 0, 0], 'a': [-1, 0, 0], 'd': 4}, {'p': [2, 0, 0], 'v': [-2, 0, 0], 'a': [-2, 0, 0], 'd': 2}]
Step: 1
[{'p': [4, 0, 0], 'v': [0, 0, 0], 'a': [-1, 0, 0], 'd': 4}, {'p': [-2, 0, 0], 'v': [-4, 0, 0], 'a': [-2, 0, 0], 'd': 2}]
Step: 2
[{'p': [3, 0, 0], 'v': [-1, 0, 0], 'a': [-1, 0, 0], 'd': 3}, {'p': [-8, 0, 0], 'v': [-6, 0, 0], 'a': [-2, 0, 0], 'd': 8}]


In [158]:
with open("input.txt") as f:
    particle_file = f.read().split("\n")

In [159]:
particles = [particle_struct(x) for x in particle_file]

In [123]:
import pandas as pd

In [134]:
df = pd.DataFrame(particles)
df.head(10)

,a,p,v
0,"[-7, 8, -10]","[1199, -2918, 1457]","[-13, 115, -8]"
1,"[-6, -5, 6]","[2551, 2418, -1471]","[-106, -108, 39]"
2,"[-6, 2, -9]","[-73, 1626, 1321]","[58, -118, -8]"
3,"[3, 3, 11]","[-3297, -894, -551]","[183, 31, -61]"
4,"[7, -12, -1]","[-1425, 4298, 617]","[32, -166, -32]"
5,"[10, -6, -4]","[-3718, 877, 1043]","[114, 7, -20]"
6,"[7, 0, -7]","[-2089, -239, 1142]","[52, 12, 3]"
7,"[-6, -3, -16]","[782, 994, 3311]","[16, -28, -32]"
8,"[7, -3, 2]","[-937, 2596, -109]","[-12, -117, -13]"
9,"[-1, 9, 14]","[-865, -2930, -1387]","[60, 76, -56]"


In [135]:
def manhattan_dist(some_list):
    abs_list = [abs(x) for x in some_list]
    return(sum(abs_list))

In [137]:
df['d'] = df['p'].map(manhattan_dist)
df.head(5)

,a,p,v,d
0,"[-7, 8, -10]","[1199, -2918, 1457]","[-13, 115, -8]",5574
1,"[-6, -5, 6]","[2551, 2418, -1471]","[-106, -108, 39]",6440
2,"[-6, 2, -9]","[-73, 1626, 1321]","[58, -118, -8]",3020
3,"[3, 3, 11]","[-3297, -894, -551]","[183, 31, -61]",4742
4,"[7, -12, -1]","[-1425, 4298, 617]","[32, -166, -32]",6340


In [139]:
for i in range(1000):
    [update_particle(x) for x in particles]

In [140]:
df1000 = pd.DataFrame(particles)

In [153]:
df1000.dtypes

a    object
d     int64
p    object
v    object
dtype: object

In [157]:
df1000.loc[df1000.d == df1000.d.min()]

,a,d,p,v
243,"[0, 0, -2]",1093388,"[-19553, -38135, -1035700]","[-20, -39, -2036]"


The first half of this puzzle is complete! It provides one gold star: *

--- Part Two ---

To simplify the problem further, the GPU would like to remove any particles that **collide**. Particles collide if their positions ever **exactly match**. Because particles are updated simultaneously, **more than two particles** can collide at the same time and place. Once particles collide, they are removed and cannot collide with anything else after that tick.

For example:
```
p=<-6,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=<-4,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
p=<-2,0,0>, v=< 1,0,0>, a=< 0,0,0>    (0)   (1)   (2)            (3)
p=< 3,0,0>, v=<-1,0,0>, a=< 0,0,0>

p=<-3,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=<-2,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
p=<-1,0,0>, v=< 1,0,0>, a=< 0,0,0>             (0)(1)(2)      (3)   
p=< 2,0,0>, v=<-1,0,0>, a=< 0,0,0>

p=< 0,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=< 0,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
p=< 0,0,0>, v=< 1,0,0>, a=< 0,0,0>                       X (3)      
p=< 1,0,0>, v=<-1,0,0>, a=< 0,0,0>

------destroyed by collision------    
------destroyed by collision------    -6 -5 -4 -3 -2 -1  0  1  2  3
------destroyed by collision------                      (3)         
p=< 0,0,0>, v=<-1,0,0>, a=< 0,0,0>
```

In this example, particles 0, 1, and 2 are simultaneously destroyed at the time and place marked X. On the next tick, particle 3 passes through unharmed.

How many particles are left after all collisions are resolved?

In [162]:
particles = [particle_struct(x) for x in particle_file]

In [163]:
particle_df = pd.DataFrame(particles)

In [168]:
particle_df['position'] = particle_df.p.apply(str)

In [170]:
particle_df.head()

,a,p,v,position
0,"[-7, 8, -10]","[1199, -2918, 1457]","[-13, 115, -8]","[1199, -2918, 1457]"
1,"[-6, -5, 6]","[2551, 2418, -1471]","[-106, -108, 39]","[2551, 2418, -1471]"
2,"[-6, 2, -9]","[-73, 1626, 1321]","[58, -118, -8]","[-73, 1626, 1321]"
3,"[3, 3, 11]","[-3297, -894, -551]","[183, 31, -61]","[-3297, -894, -551]"
4,"[7, -12, -1]","[-1425, 4298, 617]","[32, -166, -32]","[-1425, 4298, 617]"


In [184]:
particle_df.groupby('position').count() > 1

,a,p,v
position,,,
"[-1004, 863, 12]",False,False,False
"[-1006, 622, -59]",False,False,False
"[-1016, -769, -52]",False,False,False
"[-1023, 2364, -1752]",False,False,False
"[-1031, 2489, -962]",False,False,False
"[-104, 2751, 2326]",False,False,False
"[-1044, -249, 496]",False,False,False
"[-1052, 646, 3333]",False,False,False
"[-1062, -87, -642]",False,False,False


In [189]:
positions = [p.get('p') for p in particles]
positions[0:10]

[[1199, -2918, 1457],
 [2551, 2418, -1471],
 [-73, 1626, 1321],
 [-3297, -894, -551],
 [-1425, 4298, 617],
 [-3718, 877, 1043],
 [-2089, -239, 1142],
 [782, 994, 3311],
 [-937, 2596, -109],
 [-865, -2930, -1387]]

In [225]:
def find_collisions(particles):
    dup_pos = []
    positions = [p.get('p') for p in particles]
    for i in range(len(positions)):
        remain_pos = positions[:i] + positions[i+1:]
        if  positions[i] in remain_pos and positions[i] not in dup_pos:
            dup_pos.append(positions[i])
    return dup_pos

In [226]:
find_collisions([{'p':[0,0,0]}, {'p':[0,0,0]}, {'p':[0,0,0]}, {'p':[0,1,0]}, {'p':[0,1,0]}, {'p':[0,0,1]}])

[[0, 0, 0], [0, 1, 0]]

In [227]:
def remove_collisions(particles, collisions):
    particles = [p for p in particles if p.get('p') not in collisions]
    return particles

In [228]:
remove_collisions([{'p':[0,0,0]}, {'p':[0,0,0]}, {'p':[0,0,0]}, {'p':[0,1,0]}, {'p':[0,1,0]}, {'p':[0,0,1]}],
                 find_collisions([{'p':[0,0,0]}, {'p':[0,0,0]}, {'p':[0,0,0]}, {'p':[0,1,0]}, {'p':[0,1,0]}, {'p':[0,0,1]}]))

[{'p': [0, 0, 1]}]

In [244]:
pt_2_tests = [
    "p=<-6,0,0>, v=<3,0,0>, a=<0,0,0>",
    "p=<-4,0,0>, v=<2,0,0>, a=<0,0,0>",
    "p=<-2,0,0>, v=<1,0,0>, a=<0,0,0>",
    "p=<3,0,0>, v=<-1,0,0>, a=<0,0,0>",
]

pt2_tests = [particle_struct(x) for x in pt_2_tests]
pt2_tests

[{'a': [0, 0, 0], 'p': [-6, 0, 0], 'v': [3, 0, 0]},
 {'a': [0, 0, 0], 'p': [-4, 0, 0], 'v': [2, 0, 0]},
 {'a': [0, 0, 0], 'p': [-2, 0, 0], 'v': [1, 0, 0]},
 {'a': [0, 0, 0], 'p': [3, 0, 0], 'v': [-1, 0, 0]}]

In [245]:
pt2_test_particles = [x for x in pt2_tests]
for i in range(3):
    pt2_test_particles = [update_particle(x) for x in pt2_test_particles]
    collisions = find_collisions(pt2_test_particles)
    pt2_test_particles = remove_collisions(pt2_test_particles, collisions)
print("\nFinal:", len(pt2_test_particles))


Final: 1


In [252]:
updated_particles = [particle_struct(x) for x in particle_file]
cur_len = len(updated_particles)
for i in range(1000):
    updated_particles = [update_particle(x) for x in updated_particles]
    collisions = find_collisions(updated_particles)
    updated_particles = remove_collisions(updated_particles, collisions)
    if len(updated_particles) < cur_len:
        cur_len = len(updated_particles)
        print("After", i, ":", cur_len)

After 9 : 993
After 11 : 983
After 12 : 977
After 13 : 964
After 14 : 947
After 15 : 932
After 16 : 922
After 17 : 911
After 18 : 909
After 19 : 878
After 20 : 851
After 21 : 840
After 22 : 823
After 23 : 802
After 24 : 794
After 26 : 778
After 27 : 769
After 28 : 766
After 29 : 756
After 30 : 746
After 31 : 727
After 32 : 709
After 33 : 703
After 34 : 701
After 35 : 691
After 37 : 674
After 38 : 648


That's the right answer! You are one gold star closer to debugging the printer. You got rank 848 on this star's leaderboard.
